In [1]:
using Base.Dates
using AxisArrays
using Vega

In [2]:
function gen_days(nyears)
    alldays = [Date(now()-Year(nyears)):Date(now());]
    daynums = [dayofweek(d) for d in alldays]
    #weekdays = alldays[daynums .< 6]
    weekdays = alldays # there's no easy way to have weekdays as axis of an AxisArray
    #ndays = length(weekdays)

    ## delete random holidays (15 per year)
    #holidays = randperm(ndays)[1:(15 * nyears)]
    #daymask = trues(ndays)
    #daymask[holidays] = false
    #weekdays[daymask]
end

function gen_volatility(dayavgs, volatility)
    ndays = size(dayavgs, 2)
    out = copy(dayavgs)
    for d in 1:ndays
        out[:,d] += out[:,d] .* volatility .* randn(500)
    end
    abs(out)
end

function gen_prices(nstocks, ndays)
    t1prices = repmat([50.:4.3:650.;], 4)[randperm(nstocks)]
    t2prices = repmat([-50.:8.7:800.;], 6)[randperm(nstocks)]
    incrprices = (t2prices .- t1prices) / ndays

    dayavgs = Array(Float64, nstocks, ndays)
    dayavgs[:,1] = t1prices
    for d in 2:ndays
        prevday = dayavgs[:,d-1]
        dayavgs[:,d] = prevday .+ incrprices
    end
    dayavgs
end

function generate_data()
    dateaxis = Axis{:date}(Date(now()-Year(10)):Date(now()))
    scriptaxis = Axis{:script}(1:500)
    priceaxis = Axis{:price}([:open, :high, :low, :close])

    ndays = length(dateaxis)
    nstocks = length(scriptaxis)
    dayavgs = gen_prices(nstocks, ndays)
    volatility = repmat([0.0:0.5:20.0;], 30)[1:nstocks] / 100

    opens = gen_volatility(dayavgs, volatility)
    closes = gen_volatility(dayavgs, volatility)
    highs = gen_volatility(dayavgs, volatility)
    lows = gen_volatility(dayavgs, volatility)
    highs = max(opens, closes, highs, lows)
    lows = min(opens, closes, highs, lows)
    
    A = AxisArray(zeros(length(scriptaxis), length(dateaxis), length(priceaxis)), scriptaxis, dateaxis, priceaxis);
    for script in 1:500
        A[script, :, :open] = opens[script, :]
        A[script, :, :close] = closes[script, :]
        A[script, :, :low] = lows[script, :]
        A[script, :, :high] = highs[script, :]
    end
    A
end

generate_data (generic function with 1 method)

In [3]:
data = generate_data();

In [4]:
function plotprices(data::AxisArray, stock, pricecol)
    prices = data[stock, :, pricecol]
    L = length(prices)
    lineplot(x=1:L, y=reshape(prices, L))
end

function plotprices(data::AxisArray, stock)
    prices = data[stock, :]
    L = length(prices)
    lineplot(x=1:L, y=reshape(prices, L))
end

plotprices (generic function with 2 methods)

In [5]:
plotprices(data, 22, :low)

In [6]:
function dailyreturns(data::AxisArray)
    d0 = data[:,1:(end-1),:close]
    d1 = lead(data[:,:,:close], Axis{:date}(Day(1)))
    AxisArray(100 * (d0 .- d1) ./ d1, d1.axes...)
end

dailyreturns (generic function with 1 method)

In [7]:
drets = dailyreturns(data)

2-dimensional AxisArray{Float64,2,...} with axes:
    :script, 1:500
    :date, 2006-05-13:1 day:2016-05-12
And data, a 500x3653 Array{Float64,2}:
   0.0243526    0.0243585   0.0243644  …    0.219613     0.220097 
   0.974968    -0.780205    0.44195        -3.58813     -2.19867  
  -0.473559     2.1769     -3.10357         0.0461889   -0.570875 
  -0.304712     0.618515    2.1047          0.0358364    0.615498 
  -3.54089      2.27052    -1.03488        -0.191437    -4.51219  
  -6.26663      2.7776      2.89404    …    4.61091      0.912305 
   8.64797     -7.09263     7.05547         5.27898     -1.13115  
   4.20112     -3.93039     6.20805        -6.31337      4.68261  
   0.890538     0.224401   -3.52647         6.83553     -6.05321  
   5.21493     -1.81916    -4.84099        -0.262558     2.30239  
   5.07848     -6.8774      1.37024    …    2.28654     -3.63692  
  -0.384621     0.0230716   9.34105        -2.11183      4.99273  
   2.3397      16.0327     -3.12809         8.025

In [8]:
plotprices(drets, 4)

In [9]:
movavg = moving(drets, Axis{:date}(Day(5)), mean);

In [10]:
plotprices(movavg, 22)

In [11]:
volat = window(movavg, TumblingWindow(Axis{:date}(Day(365*10 + 5))), std);

In [12]:
minimum(volat), median(volat), maximum(volat)

(0.00018075832079219155,3.1077314000112746,296.65283746168507)

In [13]:
mask = 10 .< volat .< 74
maskedvolat = volat[mask]
x = axisvalues(maskedvolat, Axis{:script}())
y = reshape(copy(maskedvolat.data), length(maskedvolat)) .- 10
barplot(x=x, y=y)